In [1]:
from collections import defaultdict

def readJSON(path):
  for l in open(path, 'r'):
    d = eval(l)
    h = d['hours_transformed']
    u = d['userID']
    g = d['gameID']
    yield u,g,h,d

In [2]:
data = []

for uid, gid, hour, d in readJSON("train.json"):
    data.append({"user_id":uid, "game_id":gid, "play_time":hour})

data

[{'user_id': 'u55351001',
  'game_id': 'g35322304',
  'play_time': 0.37851162325372983},
 {'user_id': 'u70666506',
  'game_id': 'g49368897',
  'play_time': 6.011227255423254},
 {'user_id': 'u18612571',
  'game_id': 'g73495588',
  'play_time': 0.2630344058337938},
 {'user_id': 'u34283088',
  'game_id': 'g68047320',
  'play_time': 3.689299160535892},
 {'user_id': 'u16220374',
  'game_id': 'g51234623',
  'play_time': 1.263034405833794},
 {'user_id': 'u01499286',
  'game_id': 'g25723374',
  'play_time': 1.4329594072761063},
 {'user_id': 'u73063505',
  'game_id': 'g58025004',
  'play_time': 4.602884408718419},
 {'user_id': 'u29223775',
  'game_id': 'g69033010',
  'play_time': 4.321928094887363},
 {'user_id': 'u44401308',
  'game_id': 'g46446145',
  'play_time': 4.008988783227255},
 {'user_id': 'u45027672',
  'game_id': 'g02903254',
  'play_time': 2.9068905956085187},
 {'user_id': 'u33908704',
  'game_id': 'g66086214',
  'play_time': 3.0531113364595623},
 {'user_id': 'u27998358',
  'game_id'

In [3]:
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

df = pd.DataFrame(data)

In [4]:
df

,user_id,game_id,play_time
0,u55351001,g35322304,0.378512
1,u70666506,g49368897,6.011227
2,u18612571,g73495588,0.263034
3,u34283088,g68047320,3.689299
4,u16220374,g51234623,1.263034
...,...,...,...
174995,u77930643,g88529278,6.638074
174996,u01497310,g89122297,3.446256
174997,u38860000,g59833534,2.678072
174998,u17742070,g03262233,2.432959


In [5]:
# Surprise 라이브러리의 Reader 객체 생성
reader = Reader(rating_scale=(df['play_time'].min(), df['play_time'].max())) 

# 데이터셋 생성
dataset = Dataset.load_from_df(df[['user_id', 'game_id', 'play_time']], reader)

In [9]:
# 학습 데이터와 테스트 데이터로 분할
trainset, testset = train_test_split(dataset, test_size=0.03, random_state=15)

# 모델 생성 및 하이퍼파라미터 설정
model = SVD(n_factors=20, n_epochs=20, reg_pu=50, reg_qi=50, lr_all=0.0047)
model.fit(trainset)

# 검증
predictions = model.test(testset)
mse = accuracy.mse(predictions, verbose=True)
print(f"최종 검증 MSE: {mse}")

MSE: 3.0941
최종 검증 MSE: 3.0941475186335046


In [11]:
from tqdm import tqdm

with open("pairs_Hours.csv", "r") as f1:
    f1.readline() # skip the first row (column names)
    
    with open("fiveteen_coin.csv", "w") as f2:
        f2.write("ID,Label\n") # insert the column names at the first row

        for line in tqdm(f1):
            row_id, uid, gid = line.strip().split(",")
            estimate = model.predict(uid, gid).est
            # estimate = user_average[uid] if uid in user_average else global_average
            f2.write(f"{row_id},{estimate}\n")

10000it [00:00, 141614.78it/s]
